### Starting jupyter env.
```bash
cd /home/jupyter/EXP-01058 && source cell2location_startup.sh
```

### Managing jupyter env.
```bash
# Activates conda env
conda activate "/home/jupyter/EXP-01058/envs/cell2loc_env"

# Deactivates conda env
conda deactivate

# Lists all jupyter kernels
jupyter kernelspec list

# Removes cell2loc_env jupyter kernel
jupyter kernelspec remove cell2loc_env exp-01058_env

# Registers environment to a kernel
python -m ipykernel install --user --name=cell2loc_env --display-name='Environment (cell2loc_env)'
```

### Transferring scRNA data for TIS05740
```bash
gcloud auth login

gsutil -o GSUtil:parallel_composite_upload_threshold=2G -o GSUtil:parallel_thread_count=16 -o GSUtil:parallel_process_count=8 -m cp -r gs://experimental-pipeline-data/EXP-00895_EXP-00904/Exp00904_TissDiss2.zip "/home/jupyter/EXP-01058"

unzip "/home/jupyter/EXP-01058/Exp00904_TissDiss2.zip"
```

### Downloading tonsil scRNA reference atlas

Tonsil scRNA data from the ["An atlas of cells in the human tonsil" paper by Massoni-Badosa et al](https://www.cell.com/immunity/pdf/S1074-7613(24)00031-1.pdf).

CellRanger output data downloaded from [Zenodo repository](https://zenodo.org/records/10373041).

```bash
# Downloading tonsil scRNA cellranger output data

cd /home/jupyter/EXP-01058/tonsil_reference_cr_outs &&
curl -O https://zenodo.org/records/10373041/files/scRNA-seq.zip?download=1 &&
unzip scRNA-seq.zip
```

```bash
# Downloading patient metadata

cd /home/jupyter/EXP-01058/tonsil_reference_cr_outs &&
curl -O https://www.cell.com/cms/10.1016/j.immuni.2024.01.006/attachment/cc6497f7-5ffe-4cf8-9a35-7c776196ea27/mmc3.xlsx
```

## Downloading Tonsil scRNA reference

Zenodo REST API Authentification Token: qgVnt8HwocUyNKLyajDyRQzAaB8jyX3S14uf4uZU4KE00aqQ7BvGC9ONVub5

```bash
cd /home/jupyter/EXP-01058/test &&
curl -o idx.annoy https://zenodo.org/records/10520081/files/idx.annoy?download=1 &&
curl -o ref.Rds https://zenodo.org/records/10520081/files/ref.Rds?download=1
```

In [1]:
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import cell2location
import seaborn as sns
import os
import pandas as pd
from functions import *

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text for PDFs

/home/jupyter/EXP-01058/envs/cell2loc_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jupyter/EXP-01058/envs/cell2loc_env/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/jupyter/EXP-01058/envs/cell2loc_env/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


#### Loading in visium HD, visium vanilla, and scRNA data

In [2]:
home_dir = "/home/jupyter"

# Setting visium HD paths
vis_hd_bin_dir_ = "EXP-01058/TIS05740-001-021/outs/binned_outputs/square_008um"
vis_hd_spatial_dir_ = "EXP-01058/TIS05740-001-021/outs/binned_outputs/square_008um/spatial"
bin_dir = os.path.join(home_dir, vis_hd_bin_dir_)
spatial_dir = os.path.join(home_dir, vis_hd_spatial_dir_)

# Creating tissue_positions.parquet to csv
df = pd.read_parquet(os.path.join(home_dir, spatial_dir, "tissue_positions.parquet")).set_index('barcode')
df.to_csv(os.path.join(home_dir, spatial_dir, "tissue_positions.csv"))

# Setting vanilla visium paths
vis_vanilla_dir_ = "EXP-01058/EXP00826_VisB_tonsil_sr_outs/outs"
vis_vanilla_dir = os.path.join(home_dir, vis_vanilla_dir_)

# Setting scRNA paths
rna_dir_ = "EXP-01058/Exp00904_TissDiss2/Exp00904_w1_Visium_NewBlock/outs/per_sample_outs/Exp00904_w1_Visium_NewBlock/count/sample_filtered_feature_bc_matrix.h5"
rna_dir = os.path.join(home_dir, rna_dir_)

# Reading in visium HD data
hd_adata = preprocess_vis_adata(sc.read_visium(bin_dir))

# Reading in visium vanilla data
vanilla_adata = preprocess_vis_adata(sc.read_visium(vis_vanilla_dir))

# Reading in scRNA data
rna_adata = sc.read_10x_h5(rna_dir)

/home/jupyter/EXP-01058/envs/cell2loc_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/jupyter/EXP-01058/envs/cell2loc_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/jupyter/EXP-01058/envs/cell2loc_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/jupyter/EXP-01058/envs/cell2loc_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/jupyter/EXP-01058/envs/cell2loc_env/lib/python3.9/site-pac

#### cell2location deconvolution and cell type mapping

In [ ]:
vis_data.obs['sample'] = list(vis_data.uns['spatial'].keys())[0]

vis_data.var['SYMBOL'] = vis_data.var_names
vis_data.var.set_index('gene_ids', drop=True, inplace=True)

vis_data.var['MT_gene'] = [gene.startswith('MT-') for gene in vis_data.var['SYMBOL']]

vis_data.obsm['MT'] = vis_data[:, vis_data.var['MT_gene'].values].X.toarray()
vis_data = vis_data[:, ~vis_data.var['MT_gene'].values]

In [ ]:
# Install 'requests' module for python
pip install requests

# Install 'axios' module for nodejs
npm install axios

import requests
r = requests.get("https://zenodo.org/api/deposit/depositions")
r.status_code
# 401
r.json()

ACCESS_TOKEN = 'RTw8uVKhOgQUPE8ronaEANADqewQqiv2UdJ5CQTjqIKJS35CGppKAcHqYNfx'
r = requests.get('https://zenodo.org/api/deposit/depositions',
                  params={'access_token': ACCESS_TOKEN})
r.status_code
# 200
r.json()
# []